In [10]:
import glob
import os

import cv2
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
w = 100
h = 100
c = 3

seed = 109
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
# When running on the CuDNN backend, two further options must be set
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [12]:
class FlowerDataset(Dataset):
    def __init__(self, data, label):
        super(FlowerDataset, self).__init__()
        self.data = data.to(device)
        self.label = label.to(device)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

In [13]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv_model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        # self.attention = SelfAttention(12*12)

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(12*12*64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 6),
        )

    def forward(self, data):
        x = self.conv_model(data)
        x = x.view(x.size()[0],-1)
        x = self.fc(x)
        return x

In [14]:
class Stat:
    def __init__(self, training, writer=None):
        self.step = 0
        self.loss = []
        self.labels = []
        self.pred_labels = []
        self.training = training
        self.writer = writer
    
    def add(self, pred, labels, loss):
        labels = labels.cpu().numpy()
        pred = pred.cpu().detach().numpy()
        pred_labels = np.argmax(pred, axis = 1)
        self.loss.append(loss)
        self.labels.extend(labels)
        self.pred_labels.extend(pred_labels)

In [15]:
def test(model, test_data_loader):
    test_stat = Stat(training=False)
    model.eval()
    with torch.no_grad():
        for batch in test_data_loader:
            data, labels = batch[0], batch[1]
            pred_outputs = model(data)
            test_stat.add(pred_outputs, labels, 0)
    return test_stat.pred_labels

In [16]:
path_test = '../TestImages/'
imgs=[]
for im in glob.glob(path_test+'/*.jpg'):
    img=cv2.imread(im)           
    img=cv2.resize(img,(w,h)) 
    imgs.append(img)  
imgs = np.asarray(imgs,np.float32)    

test_data = imgs / 255
test_label = [x for x in range(len(test_data))]

test_data = torch.FloatTensor(test_data).permute(0, 3, 1, 2)
test_label = torch.LongTensor(test_label)
test_set = FlowerDataset(test_data, test_label)
test_data_loader = DataLoader(test_set, batch_size=len(test_data), shuffle=False)

In [17]:
K = 5
models = []
for i in range(K):
    net = MyModel()
    net.load_state_dict(torch.load(f'../cloud_model/fold{i}-flower_mlp.pt', map_location=torch.device(device)))
    models.append(net)

pred_labels = []
for net in models:
    pred_label = test(net, test_data_loader)
    pred_labels.append(pred_label)
pred_labels = np.array(pred_labels).T
# 这样pred_labels的行向量为第i个样本的预测值
final_pred = []
for sample in pred_labels:
    final_pred.append(np.argmax(np.bincount(sample)))

true_label = [1, 3, 4, 0, 2, 5]

correct = [1 if pred_label==true_label[idx] else 0 for idx, pred_label in enumerate(final_pred)]
print(f'acc = {sum(correct)/len(correct)}')

acc = 1.0
